In [14]:
import inspect
S = {i for i in range(1, 6)}
T = {i * 3 for i in range(1, 4)}.union({10})

print(S, T)

{1, 2, 3, 4, 5} {9, 10, 3, 6}


In [46]:
import itertools
from typing import Any, Callable, List, Set


def test_fn_type(domain: Set[Any], codomain: Set[Any], relation: Callable[[Any], Any]) -> dict[str, bool]:
    types = {
        "injective": True,
        "surjective": True,
        "bijective": False
    }

    # check that relation is functional
    for s in domain:
        if not relation(s) in codomain:
            raise Exception("Not left-total")

    # Test injectivity
    seen_outputs = set()
    for s in domain:
        if relation(s) in seen_outputs:
            types["injective"] = False

        seen_outputs.add(relation(s))

    # Test surjectivity
    types["surjective"] = len(seen_outputs) == len(codomain)

    # Test bijectivity
    types["bijective"] = types["injective"] and types["surjective"]

    return types
        

def fn(s):
    if s < 4:
        return s * 3
    return 10

print(test_fn_type(S, T, fn))


A = {"🐈", "🐕"}
B = {"🐈🐕", "🐕🐈"}

# def make_recursive_set(base: set, constructors: List[Callable]):
#     while True:
#         # call constructor
#         for elem in base:
#             # add new elements to set
#             base = base.union({con(elem) for con in constructors})
#         yield base


def is_unary(fn: Callable):
    return len(inspect.signature(fn).parameters) == 1

def make_recursive_set(base: set, constructors: List[Callable[[Any, Any], Any]]):
    while True:
        
        # # split constructors into unary and binary ones
        # unary_cons, binary_cons = [], []
        # for con in constructors:
        #     if is_unary(con):
        #         unary_cons.append(con)
        #     else:
        #         binary_cons.append(con)
        
        # Every pair in cartesian product
        product = itertools.product(base, base)

        for x, y in product:
            new_elems = {con(x)
                        if is_unary(con)
                        else con(x, y) for con in constructors}

            base = base.union(new_elems)

        yield base

{'injective': False, 'surjective': True, 'bijective': False}


In [45]:
A = set([""])

constructed = make_recursive_set(A, [lambda x, y: f"♡{x}♣{y}"])

print(sorted(next(constructed)))
print(sorted(next(constructed)))

items = list(next(constructed))

for item in items:
    assert item.count("♡") == item.count("♣"), "Unequal length"


['', '♡♣']
['', '♡♡♣♣', '♡♡♣♣♡♣', '♡♣', '♡♣♡♣']
